## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
import h5py
from google.colab import files

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df1=application_df.drop(columns=["EIN","NAME"])
application_df1.head()



,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df1.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
application_df1['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
application_types_to_replace=application_df1['APPLICATION_TYPE'][application_df1['APPLICATION_TYPE'].isin(application_df1['APPLICATION_TYPE'].value_counts()[application_df1['APPLICATION_TYPE'].value_counts()<528].index)]

# Replace in dataframe
for app in application_types_to_replace:
    application_df1['APPLICATION_TYPE'] = application_df1['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df1['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
application_df1['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
application_df1['CLASSIFICATION'].value_counts()[application_df1['CLASSIFICATION'].value_counts()>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace=application_df1['CLASSIFICATION'][application_df1['CLASSIFICATION'].isin(application_df1['CLASSIFICATION'].value_counts()[application_df1['CLASSIFICATION'].value_counts()<1883].index)]
# Replace in dataframe
for cls in classifications_to_replace:
    application_df1['CLASSIFICATION'] = application_df1['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df1['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
application_df2=pd.get_dummies(application_df1)
application_df2.dtypes

STATUS                          int64
ASK_AMT                         int64
IS_SUCCESSFUL                   int64
APPLICATION_TYPE_Other          uint8
APPLICATION_TYPE_T10            uint8
APPLICATION_TYPE_T19            uint8
APPLICATION_TYPE_T3             uint8
APPLICATION_TYPE_T4             uint8
APPLICATION_TYPE_T5             uint8
APPLICATION_TYPE_T6             uint8
APPLICATION_TYPE_T7             uint8
APPLICATION_TYPE_T8             uint8
AFFILIATION_CompanySponsored    uint8
AFFILIATION_Family/Parent       uint8
AFFILIATION_Independent         uint8
AFFILIATION_National            uint8
AFFILIATION_Other               uint8
AFFILIATION_Regional            uint8
CLASSIFICATION_C1000            uint8
CLASSIFICATION_C1200            uint8
CLASSIFICATION_C2000            uint8
CLASSIFICATION_C2100            uint8
CLASSIFICATION_C3000            uint8
CLASSIFICATION_Other            uint8
USE_CASE_CommunityServ          uint8
USE_CASE_Heathcare              uint8
USE_CASE_Oth

In [10]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y=application_df2['IS_SUCCESSFUL'].values[:30000]
x=application_df2.drop(columns='IS_SUCCESSFUL').values[:30000]
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=43)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=90, activation="relu", input_dim=43))

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=35, activation="relu"))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 90)                3960      
                                                                 
 dense_1 (Dense)             (None, 35)                3185      
                                                                 
 dense_2 (Dense)             (None, 1)                 36        
                                                                 
Total params: 7,181
Trainable params: 7,181
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
#  YOUR CODE GOES HERE
# Define the callback
checkpoint_callback = ModelCheckpoint(
    filepath='model_weights_{epoch:02d}.h5',
    save_weights_only=True,
    period=5
)
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
704/704 [==============================] - 4s 3ms/step - loss: 0.5646 - accuracy: 0.7313
Epoch 2/50
704/704 [==============================] - 2s 4ms/step - loss: 0.5467 - accuracy: 0.7361
Epoch 3/50
704/704 [==============================] - 1s 2ms/step - loss: 0.5432 - accuracy: 0.7392
Epoch 4/50
704/704 [==============================] - 2s 2ms/step - loss: 0.5400 - accuracy: 0.7400
Epoch 5/50
704/704 [==============================] - 2s 2ms/step - loss: 0.5391 - accuracy: 0.7428
Epoch 6/50
704/704 [==============================] - 1s 2ms/step - loss: 0.5382 - accuracy: 0.7413
Epoch 7/50
704/704 [==============================] - 1s 2ms/step - loss: 0.5374 - accuracy: 0.7418
Epoch 8/50
704/704 [==============================] - 1s 2ms/step - loss: 0.5364 - accuracy: 0.7424
Epoch 9/50
704/704 [==============================] - 1s 2ms/step - loss: 0.5360 - accuracy: 0.7420
Epoch 10/50
704/704 [==============================] - 1s 2ms/step - loss: 0.5355 - accuracy: 0.7430

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

235/235 - 1s - loss: 0.5429 - accuracy: 0.7417 - 626ms/epoch - 3ms/step
Loss: 0.542941153049469, Accuracy: 0.7417333126068115


In [16]:
# Create an HDF5 file
with h5py.File('AlphabetSoupCharity.h5', 'w') as f:
    # Create a dataset to store the results
    f.create_dataset('nn', data=str(nn))

# Download the HDF5 file

files.download('AlphabetSoupCharity.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn.save('AlphabetSoupCharity.h5')